In [119]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Optional
from math import pi, atan2, cos, sin, sqrt
from typing import List, Tuple


In [120]:
@dataclass
class node2d:    
    connections: List['node2d']
    weights: List[float]
    angle_ranges: List[Tuple[float, float]]
    distance_ranges: List[Tuple[float, float]]
    is_rotating_point: bool
    position: Tuple[float, float]
    velocity: Tuple[float, float]

    def __init__(self, 
        connections: Optional[List['node2d']] = None,
        weights: Optional[List[float]] = None,
        angle_ranges: Optional[List[Tuple[float, float]]] = None,
        distance_ranges: Optional[List[Tuple[float, float]]] = None,
        position:  Optional[Tuple[float, float]] = None,
        velocity: Optional[Tuple[float, float]] = None,
    ) -> None:
        self.connections = connections
        if not connections:
            self.connections = []
        self.weights = weights
        if not weights:
            self.weights = []
        self.angle_ranges = angle_ranges
        if not angle_ranges:
            self.angle_ranges = []
        self.distance_ranges = distance_ranges
        if not distance_ranges:
            self.distance_ranges = []
        self.position = position
        if not position:
            self.position = (0.0, 0.0)
        self.velocity = velocity
        if not velocity:
            self.velocity = (0.0, 0.0)
        self.is_rotating_point = False
    
    def __hash__(self):
        return hash((self.position, self.velocity))
    
    def __repr__(self) -> str:
        return f"node2d(pos: {self.position} vel: {self.velocity})"

In [121]:
class node2d_utils:
    def dfs(start: node2d, target: node2d = None) -> List[node2d]:
        visited = set()
        stack = [(start, [start])]

        while stack:
            current, path = stack.pop()

            if current == target:
                return path

            visited.add(current)

            for neighbor in current.connections:
                if neighbor not in visited:
                    new_path = path + [neighbor]
                    stack.append((neighbor, new_path))
        return []

    @staticmethod
    def all_nodes(start: node2d) -> List[node2d]:
        visited = set()
        stack = [start]
        all_nodes = []

        while stack:
            current = stack.pop()

            if current not in visited:
                visited.add(current)
                all_nodes.append(current)

                for neighbor in current.connections:
                    if neighbor not in visited:
                        stack.append(neighbor)

        return all_nodes

    @staticmethod
    def calculate_kinematics(root: node2d, timestep: float) -> None:
        def update_node(node: node2d, parent_pos: Tuple[float, float], parent_vel: Tuple[float, float]) -> None:
            if not node.connections:
                return

            for i, (conn, weight, angle_range, dist_range) in enumerate(zip(node.connections, node.weights, node.angle_ranges, node.distance_ranges)):
                min_angle, max_angle = angle_range
                min_dist, max_dist = dist_range

                # Calculate the desired angle and distance based on the parent's position
                dx, dy = conn.position[0] - parent_pos[0], conn.position[1] - parent_pos[1]
                desired_angle = atan2(dy, dx)
                desired_dist = sqrt(dx**2 + dy**2)

                # Constrain the angle and distance within the specified ranges
                angle = max(min_angle, min(max_angle, desired_angle))
                dist = max(min_dist, min(max_dist, desired_dist))

                # Calculate the new position and velocity of the node
                node.position = (parent_pos[0] + dist * cos(angle), parent_pos[1] + dist * sin(angle))
                node.velocity = (parent_vel[0] + weight * (node.position[0] - conn.position[0]),
                                parent_vel[1] + weight * (node.position[1] - conn.position[1]))

                # Recursively update the child nodes
                update_node(conn, node.position, node.velocity)

        # Start the recursive update from the root node
        update_node(root, root.position, root.velocity)

In [133]:


n = node2d(velocity=(5, 5))
n.connections.append(node2d(position=(1, 1), velocity=(1, 1)))
print(f"{node2d_utils.all_nodes(n)}",)
for i in range(10):
    node2d_utils.calculate_kinematics(n, 0.01)
print(f"{node2d_utils.all_nodes(n)}",)
    


 [node2d(pos: (0.0, 0.0) vel: (5, 5)), node2d(pos: (1, 1) vel: (1, 1))]
[node2d(pos: (0.0, 0.0) vel: (5, 5)), node2d(pos: (1, 1) vel: (1, 1))]
